# SuperflexPy: A new open source framework for building conceptual hydrological models

Authors: Marco Dal Molin, Dmitri Kavetski, Mario Schirmer, and Fabrizio Fenicia

This is a tutorial for the implementation of the GR4J model using the SuperflexPy framework.

For reference, look at the [official guide](https://https://superflexpy.readthedocs.io/)

Framework code available on [Github](https://github.com/dalmo1991/superflexPy)


![alt text](https://github.com/dalmo1991/superflexPy/raw/master/doc/pics/GR4J.png)

## 01 - Installation

In [0]:
!pip install superflexpy

## 02 - Import the elements

In [0]:
# Elements of the framework
from superflexpy.framework.unit import Unit

# Structural elements
from superflexpy.implementation.elements.structure_elements import Splitter, Junction, Transparent

# GR4J elements
from superflexpy.implementation.elements.gr4j import InterceptionFilter, ProductionStore, UnitHydrograph1, UnitHydrograph2, RoutingStore, FluxAggregator

# Root finder
from superflexpy.implementation.computation.pegasus_root_finding import PegasusPython

## 03 - Initialize the elements

In [0]:
# Parameters of GR4J -> the others are set to the default value
x1, x2, x3, x4 = (50.0, 0.1, 20.0, 3.5)

# Root finder (used to solve the differentail equations)
solver_pegasus = PegasusPython()  # Use the default parameters

# Interception Filter
interception_filter = InterceptionFilter(id='ir')

# Production Store
production_store = ProductionStore(parameters={'x1': x1, 'alpha': 2.0, 
                                               'beta': 5.0, 'ni': 4/9},
                                   states={'S0': 10.0},
                                   solver=solver_pegasus,
                                   id='ps')

# Splitter
splitter = Splitter(weight=[[0.9], [0.1]], 
                    direction=[[0], [0]],
                    id='spl')

# Unit Hydrograph 1
unit_hydrograph_1 = UnitHydrograph1(parameters={'lag-time': x4},
                                    states={'lag': None},
                                    id='uh1')

# Unit Hydrograph 2
unit_hydrograph_2 = UnitHydrograph2(parameters={'lag-time': 2*x4},
                                    states={'lag': None},
                                    id='uh2')

# Routing Store
routing_store = RoutingStore(parameters={'x2': x2, 'x3': x3,
                                         'gamma': 5.0, 'omega': 3.5},
                             states={'S0': 10.0},
                             solver=solver_pegasus,
                             id='rs')

# Transparent Element
transparent = Transparent(id='tr') 

# Junction
junction = Junction(direction=[[0, None],  # First output
                               [1, None],  # Second output
                               [None, 0]], # Third output
                    id='jun')

# Flux Aggregator
flux_aggregator = FluxAggregator(id='fa')

# Build the unit
structure = Unit(layers=[[interception_filter],
                         [production_store],
                         [splitter],
                         [unit_hydrograph_1, unit_hydrograph_2],
                         [routing_store, transparent],
                         [junction],
                         [flux_aggregator]],
                 id='structure')

## 04 - Play with the parameters and the states

Once the elements have been initialized, the parameters and the states can be updated. Note how the identifiers of the element and of the unit are added to the original name of the parameter or state.

In [0]:
# Get all the parameters
print(structure.get_parameters())

{'structure_ps_x1': 50.0, 'structure_ps_alpha': 2.0, 'structure_ps_beta': 5.0, 'structure_ps_ni': 0.4444444444444444, 'structure_uh1_lag-time': 3.5, 'structure_uh2_lag-time': 7.0, 'structure_rs_x2': 0.1, 'structure_rs_x3': 20.0, 'structure_rs_gamma': 5.0, 'structure_rs_omega': 3.5}


In [0]:
# Change the parameter 'structure_ps_x1'

structure.set_parameters(parameters={'structure_ps_x1' : 45.0})

# Notice how the elements are copied into the structure and not linked: the
# parameter has changed only in the structure and not in the element.

print(production_store.get_parameters())
print(structure.get_parameters())

{'ps_x1': 50.0, 'ps_alpha': 2.0, 'ps_beta': 5.0, 'ps_ni': 0.4444444444444444}
{'structure_ps_x1': 45.0, 'structure_ps_alpha': 2.0, 'structure_ps_beta': 5.0, 'structure_ps_ni': 0.4444444444444444, 'structure_uh1_lag-time': 3.5, 'structure_uh2_lag-time': 7.0, 'structure_rs_x2': 0.1, 'structure_rs_x3': 20.0, 'structure_rs_gamma': 5.0, 'structure_rs_omega': 3.5}


## 05 - Run the structure

In [0]:
# Generate inputs
import numpy as np

NUM_TIMESTEPS = 100

# Create input fluxes
precipitation = np.random.rand(NUM_TIMESTEPS)*10  # Precipitation input
precipitation[10:20] = 0
precipitation[55:60] = 0
precipitation[70:] = 0
pet = np.array([2.0]*NUM_TIMESTEPS)

# Set the timestep
structure.set_timestep(dt=1.0)

# Set the inputs
structure.set_input([pet, precipitation])

# Get the output
output = structure.get_output(solve=True)

# Notice that now the states of the storages are the ones at the ones at the
# end of the simulation. If you run the model again, it will start from those
# states.

print(structure.get_states())

{'structure_ps_S0': 4.944337435652198, 'structure_uh1_lag': [array([9.91303081e-06, 5.76975894e-06, 2.02315437e-06, 0.00000000e+00])], 'structure_uh2_lag': [array([1.77692002e-06, 1.14351078e-06, 6.63589307e-07, 2.93493533e-07,
       9.43696994e-08, 1.53355828e-08, 0.00000000e+00])], 'structure_rs_S0': 8.658425007791461}


## 06 - Inspect after running

Note that when calling the method `get_output` we need to pass the argument `solve=False`

In [0]:
# Interception
output_interception = structure.call_internal(id='ir', method='get_output', solve=False)

# Production
output_production = structure.call_internal(id='ps', method='get_output', solve=False)
aet_production = structure.call_internal(id='ps', method='get_aet')
state_production= structure.get_internal(id='ps', attribute='state_array')

# Unit hydrographs
output_uh1 = structure.call_internal(id='uh1', method='get_output', solve=False)
output_uh2 = structure.call_internal(id='uh2', method='get_output', solve=False)

# Routing
output_routing = structure.call_internal(id='rs', method='get_output', solve=False)
state_routing= structure.get_internal(id='rs', attribute='state_array')

## 07 - Plot

Click on the legend to turn on/off the traces

In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.2)

fig.add_trace(trace=go.Bar(x=list(range(NUM_TIMESTEPS)), y=precipitation, name='precipitation'),
              row=1, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output[0], name='Final Output'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_interception[0], name='Interception_PET'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_interception[1], name='Interception_P'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_production[0], name='Production_Output'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=aet_production, name='Production_AET'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=state_production[:, 0], name='Production_Storage'),
              row=3, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_uh1[0], name='UH1_Output'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_uh2[0], name='UH2_Output'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_routing[0], name='Routing_Output'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=output_routing[1], name='Loss'),
              row=2, col=1)

fig.add_trace(trace=go.Scatter(x=list(range(NUM_TIMESTEPS)), y=state_routing[:, 0], name='Routing_Storage'),
              row=3, col=1)

fig.update_xaxes(title_text="Timestep", row=1, col=1)
fig.update_yaxes(title_text="Precipitation [mm/timestep]", row=1, col=1)
fig.update_xaxes(title_text="Timestep", row=2, col=1)
fig.update_yaxes(title_text="Fluxes [mm/timestep]", row=2, col=1)
fig.update_xaxes(title_text="Timestep", row=3, col=1)
fig.update_yaxes(title_text="Storage [mm]", row=3, col=1)
fig.update_layout(height=750)